In [1]:
import os
os.chdir("../")
%pwd

'/home/dheeraj37/Projects/Audio-Genre-Classification-1'

In [2]:
import tensorflow as tf
from dataclasses import dataclass
from pathlib import Path


2023-09-24 00:43:16.892134: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-09-24 00:43:16.893246: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-24 00:43:16.915394: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-24 00:43:16.916005: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-24 00:43:17.409884: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not fin

In [3]:
model = tf.keras.models.load_model("artifacts/model_training/model.h5")

2023-09-24 00:43:18.613365: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-24 00:43:18.613584: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [4]:
@dataclass(frozen=True)
class EvaluationConfig:
    model_path: Path
    test_data_path: Path
    all_params: dict
    params_image_size: list
    params_batch_size: int

In [5]:
from cnnClassifier.constant import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [6]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_PATH,
        params_filepath = PARAMS_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            model_path="artifacts/model_training/model.h5",
            test_data_path="artifacts/prepare_data/test_data.json",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config


In [7]:
from urllib.parse import urlparse
from cnnClassifier.utils.common import MFCCDataGenerator
import pandas as pd

In [10]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    def get_test_data(self):
        self.test_df = pd.read_json(self.config.test_data_path)


    def test_generator(self):
        self.test_data_generator = MFCCDataGenerator(
            self.test_df, batch_size=32, shuffle=False)

    
    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.model_path)
        self.test_generator()
        self.score = model.evaluate(self.test_data_generator)

    
    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)  



In [11]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.get_test_data()
    evaluation.evaluation()
    evaluation.save_score()

except Exception as e:
   raise e


[2023-09-24 00:44:20,780]: INFO: common: yaml file: config/config.yaml loaded successfully
[2023-09-24 00:44:20,782]: INFO: common: yaml file: params.yaml loaded successfully
[2023-09-24 00:44:20,783]: INFO: common: created directory at: artifacts
29/29 [==============================] - 0s 7ms/step - loss: 1.4440 - accuracy: 0.6600
[2023-09-24 00:44:21,533]: INFO: common: json file saved at: scores.json
